In [1]:
from dataclasses import dataclass, replace
from math import prod
from parse import parse
import aocd

In [2]:
@dataclass(frozen=True)
class Blueprint:
    id: int
    ore_for_ore: int
    ore_for_clay: int
    ore_for_obsidian: int
    clay_for_obsidian: int
    ore_for_geode: int
    obsidian_for_geode: int

@dataclass(unsafe_hash=True)
class RobotFactory:
    blueprint: Blueprint
    time_left: int
    ore: int
    clay: int
    obsidian: int
    geodes: int
    ore_robots: int
    clay_robots: int
    obsidian_robots: int
    geode_robots: int
    building_ore_robot: bool
    building_clay_robot: bool
    building_obsidian_robot: bool
    building_geode_robot: bool

    def collect(self):
        self.ore += self.ore_robots
        self.clay += self.clay_robots
        self.obsidian += self.obsidian_robots
        self.geodes += self.geode_robots
    
    def can_build_robot(self, robot_type):
        match robot_type:
            case 'ore':
                return self.ore >= self.blueprint.ore_for_ore
            case 'clay':
                return self.ore >= self.blueprint.ore_for_clay
            case 'obsidian':
                return self.ore >= self.blueprint.ore_for_obsidian and self.clay >= self.blueprint.clay_for_obsidian
            case 'geode':
                return self.ore >= self.blueprint.ore_for_geode and self.obsidian >= self.blueprint.obsidian_for_geode
    
    def start_building_robot(self, robot_type):
        match robot_type:
            case 'ore':
                self.ore -= self.blueprint.ore_for_ore
                self.building_ore_robot = True
            case 'clay':
                self.ore -= self.blueprint.ore_for_clay
                self.building_clay_robot = True
            case 'obsidian':
                self.ore -= self.blueprint.ore_for_obsidian
                self.clay -= self.blueprint.clay_for_obsidian
                self.building_obsidian_robot = True
            case 'geode':
                self.ore -= self.blueprint.ore_for_geode
                self.obsidian -= self.blueprint.obsidian_for_geode
                self.building_geode_robot = True
    
    def finish_building_robots(self):
        if self.building_ore_robot:
            self.ore_robots += 1
            self.building_ore_robot = False
        if self.building_clay_robot:
            self.clay_robots += 1
            self.building_clay_robot = False
        if self.building_obsidian_robot:
            self.obsidian_robots += 1
            self.building_obsidian_robot = False
        if self.building_geode_robot:
            self.geode_robots += 1
            self.building_geode_robot = False

In [3]:
def parse_blueprint(line):
    assert (m := parse('Blueprint {id:d}: Each ore robot costs {ore_for_ore:d} ore. Each clay robot costs {ore_for_clay:d} ore. Each obsidian robot costs {ore_for_obsidian:d} ore and {clay_for_obsidian:d} clay. Each geode robot costs {ore_for_geode:d} ore and {obsidian_for_geode:d} obsidian.', line))
    return Blueprint(**m.named)

In [4]:
def time_step(factory_set: set[RobotFactory]):
    factory_list = list(factory_set)
    new_factories = []
    for factory in factory_list:
        for robot_type in ['ore', 'clay', 'obsidian', 'geode']:
            if factory.can_build_robot(robot_type):
                new_factory = replace(factory)
                new_factory.start_building_robot(robot_type)
                new_factories.append(new_factory)
    factory_list.extend(new_factories)

    for factory in factory_list:
        factory.collect()
        factory.finish_building_robots()
        factory.time_left -= 1
    
    return factory_list

In [5]:
lines = aocd.get_data(day=19, year=2022).splitlines()
blueprints = [parse_blueprint(line) for line in lines]

In [6]:
part1_results = []
for blueprint in blueprints:
    first_factory = RobotFactory(blueprint, 24, 0, 0, 0, 0, 1, 0, 0, 0, False, False, False, False)
    factory_set = set([first_factory])

    for i in range(24):
        factories = time_step(factory_set)
        max_geodes = max(f.geodes for f in factories)
        max_geode_robots = max(f.geode_robots for f in factories)
        factory_set = set(f for f in factories if f.geodes == max_geodes or f.geode_robots == max_geode_robots)
    max_geodes = max(f.geodes for f in factory_set)
    part1_results.append((blueprint.id, max_geodes))
    print(f"{blueprint.id}\t{max_geodes}")

print("Part 1:", sum(bpid * geodes for (bpid, geodes) in part1_results))

1	2
2	1
3	0
4	8
5	1
6	11
7	0
8	2
9	0
10	1
11	1
12	7
13	0
14	0
15	0
16	2
17	2
18	7
19	2
20	6
21	5
22	1
23	1
24	1
25	0
26	0
27	0
28	1
29	2
30	5
Part 1: 988


In [8]:
part2_results = []
for blueprint in blueprints[:3]:
    first_factory = RobotFactory(blueprint, 24, 0, 0, 0, 0, 1, 0, 0, 0, False, False, False, False)
    factory_set = set([first_factory])

    for i in range(32):
        factories = time_step(factory_set)
        max_geodes = max(f.geodes for f in factories)
        max_geode_robots = max(f.geode_robots for f in factories)
        factory_set = set(f for f in factories if f.geodes == max_geodes or f.geode_robots == max_geode_robots)
    max_geodes = max(f.geodes for f in factory_set)
    part2_results.append(max_geodes)
    print(f"{blueprint.id}\t{max_geodes}")

print("Part 2:", prod(part2_results))

1	30
2	26
3	11
Part 2: 8580
